In [ ]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 71.3 MB/s eta 0:00:00


In [ ]:
# 1-1. 데이터 세그멘타이즈 함수

def segment_epoch(data, epoch_len_sec=30, segment_len_sec=0.5, sfreq=100):
    total_samples = data.shape[1]
    epoch_len = int(epoch_len_sec * sfreq)     # 예: 3000 samples (30초)
    segment_len = int(segment_len_sec * sfreq) # 예: 50 samples (0.5초)

    num_epochs = total_samples // epoch_len
    all_epochs = []

    for i in range(num_epochs):
        epoch = data[:, i*epoch_len:(i+1)*epoch_len]  # shape: (채널, 3000)
        # 채널별로 segment 단위 나누기
        eeg = epoch[0].reshape(-1, segment_len)  # (60, 50) if 30sec/0.5sec=60 segments
        eog = epoch[1].reshape(-1, segment_len)
        all_epochs.append((eeg, eog))  # 각 epoch당 (EEG, EOG) 튜플
    return all_epochs

In [ ]:
# 1-2. 라벨 변환 함수
def convert_to_epoch_labels(annotations, epoch_sec=30, total_epochs=None):
    label_map = {
        'Sleep stage W': 0,
        'Sleep stage 1': 1,
        'Sleep stage 2': 2,
        'Sleep stage 3': 3,
        'Sleep stage 4': 3,
        'Sleep stage R': 4,
        'Sleep stage ?': -1,
        'Movement time': -1
    }
    labels = []
    for desc, dur in zip(annotations.description, annotations.duration):
        n_epochs = int(dur // epoch_sec)
        labels.extend([label_map.get(desc, -1)] * n_epochs)
    if total_epochs is not None:
        labels = labels[:total_epochs]
    return labels

In [ ]:
import os
from glob import glob
import re
import pickle
import mne

def extract_id(fname):
    m = re.search(r"SC(\d{4})", os.path.basename(fname))
    return m.group(1) if m else None

def save_pickle(data, labels, part_idx):
    save_path = f"/content/drive/MyDrive/sleep_data_part{part_idx}.pkl"
    with open(save_path, 'wb') as f:
        pickle.dump((data, labels), f)
    print(f"✅ Saved: {save_path}")

def load_full_dataset_by_chunks(psg_dir, chunk_size=20, start_chunk=0, channels=['EEG Fpz-Cz', 'EOG horizontal'], epoch_len_sec=30, segment_len_sec=0.5):
    psg_files = sorted(glob(os.path.join(psg_dir, "*-PSG.edf")))
    hypno_files = sorted(glob(os.path.join(psg_dir, "*-Hypnogram.edf")))

    psg_dict = {extract_id(f): f for f in psg_files if extract_id(f)}
    hypno_dict = {extract_id(f): f for f in hypno_files if extract_id(f)}

    ids = sorted(set(psg_dict.keys()) & set(hypno_dict.keys()))
    total_chunks = (len(ids) + chunk_size - 1) // chunk_size

    for chunk_idx in range(start_chunk, total_chunks):
        chunk_ids = ids[chunk_idx * chunk_size : (chunk_idx + 1) * chunk_size]
        print(f"\n🚀 Processing chunk {chunk_idx+1}/{total_chunks} (IDs: {chunk_ids})")

        all_segmented = []
        all_labels = []

        for file_id in chunk_ids:
            psg_path = psg_dict[file_id]
            hypno_path = hypno_dict[file_id]
            print(f"   🔄 {file_id} — PSG: {os.path.basename(psg_path)}, Hypnogram: {os.path.basename(hypno_path)}")

            try:
                raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
                raw.set_annotations(mne.read_annotations(hypno_path))
                raw.pick_channels(channels)

                data = raw.get_data()
                sfreq = raw.info['sfreq']
                segmented = segment_epoch(data, epoch_len_sec, segment_len_sec, sfreq)
                labels = convert_to_epoch_labels(raw.annotations, epoch_sec=epoch_len_sec, total_epochs=len(segmented))

                filtered = [(seg, lab) for seg, lab in zip(segmented, labels) if lab != -1]
                if not filtered:
                    continue

                seg_data, seg_labels = zip(*filtered)
                all_segmented.extend(seg_data)
                all_labels.extend(seg_labels)

            except Exception as e:
                print(f"‼️ Failed to load {file_id}: {e}")
                continue

        save_pickle(all_segmented, all_labels, chunk_idx + 1)

# 사용 예시
psg_dir = "/content/drive/MyDrive/Colab Notebooks/sleep-cassette"
load_full_dataset_by_chunks(psg_dir, chunk_size=20, start_chunk=0)


🚀 Processing chunk 1/8 (IDs: ['4001', '4002', '4011', '4012', '4021', '4022', '4031', '4032', '4041', '4042', '4051', '4052', '4061', '4062', '4071', '4072', '4081', '4082', '4091', '4092'])
   🔄 4001 — PSG: SC4001E0-PSG.edf, Hypnogram: SC4001EC-Hypnogram.edf


<ipython-input-5-15075ba00614>:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<ipython-input-5-15075ba00614>:41: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(mne.read_annotations(hypno_path))
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)


   🔄 4002 — PSG: SC4002E0-PSG.edf, Hypnogram: SC4002EC-Hypnogram.edf
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

   🔄 4822 — PSG: SC4822G0-PSG.edf, Hypnogram: SC4822GC-Hypnogram.edf


<ipython-input-5-15075ba00614>:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
<ipython-input-5-15075ba00614>:40: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<ipython-input-5-15075ba00614>:41: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(mne.read_annotations(hypno_path))


✅ Saved: /content/drive/MyDrive/sleep_data_part8.pkl


In [ ]:
# 파이토치 데이터셋 클래스 정의
class SleepEDF_EpochDataset(Dataset):
    def __init__(self, segmented_data, labels, normalize=True):
        """
        segmented_data: list of (eeg, eog) 각 shape: (60, 50)
        labels: list of integer 라벨 (0~4)
        """
        self.data = segmented_data
        self.labels = labels
        self.normalize = normalize

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        eeg, eog = self.data[idx]  # (60, 50), (60, 50)
        label = self.labels[idx]

        if self.normalize:
            eeg = (eeg - eeg.mean()) / (eeg.std() + 1e-6)
            eog = (eog - eog.mean()) / (eog.std() + 1e-6)

        # (1, 60, 50) 형태로 변환
        eeg = torch.tensor(eeg, dtype=torch.float32).unsqueeze(0)
        eog = torch.tensor(eog, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor(label, dtype=torch.long)
        return eeg, eog, label

# 데이터셋 생성
dataset = SleepEDF_EpochDataset(all_segmented_data, all_labels_data)
print("데이터셋 생성 완료:", len(dataset), "epochs")

### 데이터 만들기 끝

## pkl 파일 로드

In [ ]:
import torch
import pickle
from sklearn.model_selection import train_test_split
import torch
data = torch.load('/content/drive/MyDrive/Colab Notebooks/sleep_classification_checkpoint.pkl', map_location='cpu')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/sleep_classification_checkpoint.pkl'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pickle

## 모델 정의

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Channel Attention
class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.mlp(x).unsqueeze(2)
        return x * scale

# EEG/EOG CNN Feature Extractor
class SignalCNN(nn.Module):
    def __init__(self, in_channels=1, base_channels=32):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(in_channels, base_channels, kernel_size=5, padding=2),
            nn.BatchNorm1d(base_channels),
            nn.PReLU(),
            ChannelGate(base_channels),
            nn.MaxPool1d(2),

            nn.Conv1d(base_channels, base_channels * 2, kernel_size=5, padding=2),
            nn.BatchNorm1d(base_channels * 2),
            nn.PReLU(),
            ChannelGate(base_channels * 2),
            nn.MaxPool1d(2),
        )

    def forward(self, x):
        return self.block(x)  # (B, 64, L)

# Cross-Modal Attention
class CrossModalAttention(nn.Module):
    def __init__(self, dim, heads=2):
        super().__init__()
        self.query_proj = nn.Linear(dim, dim)
        self.key_proj = nn.Linear(dim, dim)
        self.value_proj = nn.Linear(dim, dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=heads, batch_first=True)

    def forward(self, eeg, eog):
        Q = self.query_proj(eeg)
        K = self.key_proj(eog)
        V = self.value_proj(eog)
        out, _ = self.attn(Q, K, V)
        return out

# Temporal Convolutional Network (TCN)
class TCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1):
        super().__init__()
        padding = (kernel_size - 1) * dilation // 2
        self.block = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        return self.block(x)

class TCN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=3):
        super().__init__()
        layers = []
        for i in range(num_layers):
            dilation = 2 ** i
            layers.append(TCNBlock(input_dim if i == 0 else hidden_dim, hidden_dim, dilation=dilation))
        self.network = nn.Sequential(*layers)

    def forward(self, x):  # x: (B, L, C)
        x = x.transpose(1, 2)  # → (B, C, L)
        x = self.network(x)
        return x.transpose(1, 2)  # → (B, L, C)

# Attention Pooling + Classifier
class AttnClassifier(nn.Module):
    def __init__(self, input_dim, num_classes=5):
        super().__init__()
        self.attn = nn.Linear(input_dim, 1)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        attn_weights = torch.softmax(self.attn(x), dim=1)  # (B, L, 1)
        pooled = torch.sum(attn_weights * x, dim=1)        # (B, C)
        out = self.fc(pooled)
        return out, attn_weights

# Full CSTAN-TCN Model
class CSTAN_TCN(nn.Module):
    def __init__(self, cnn_dim=32, tcn_dim=64, num_classes=5):
        super().__init__()
        self.eeg_cnn = SignalCNN(1, cnn_dim)
        self.eog_cnn = SignalCNN(1, cnn_dim)
        self.cross_attn = CrossModalAttention(dim=cnn_dim * 2)
        self.tcn = TCN(input_dim=cnn_dim * 2, hidden_dim=tcn_dim)
        self.classifier = AttnClassifier(input_dim=tcn_dim, num_classes=num_classes)

    def forward(self, x):  # x: (B, 2, T)
        eeg = x[:, 0:1, :]  # (B, 1, T)
        eog = x[:, 1:2, :]  # (B, 1, T)

        eeg_feat = self.eeg_cnn(eeg)  # (B, C, L)
        eog_feat = self.eog_cnn(eog)

        eeg_feat = eeg_feat.transpose(1, 2)  # (B, L, C)
        eog_feat = eog_feat.transpose(1, 2)

        fused = self.cross_attn(eeg_feat, eog_feat)  # (B, L, C)
        tcn_out = self.tcn(fused)  # (B, L, C)

        out, attn = self.classifier(tcn_out)
        return out, attn

## 데이터 합치기

In [ ]:
import os
import pickle
import torch
import numpy as np

# 병합할 pkl 경로 설정
pkl_dir = "/content/drive/MyDrive/sleep_segments"
file_list = [os.path.join(pkl_dir, f"sleep_data_part{i}.pkl") for i in range(1, 7)]

all_segments, all_labels = [], []

print("🔄 피클 파일 병합 중...")
for file in file_list:
    with open(file, 'rb') as f:
        segments, labels = pickle.load(f)
        for seg in segments:
            tensor_seg = torch.tensor(seg, dtype=torch.float32)
            if tensor_seg.ndim == 3:  # (2, 60, 50)
                tensor_seg = tensor_seg.view(2, -1)  # (2, 3000)
            all_segments.append(tensor_seg)
        all_labels.extend(labels)

all_segments = torch.stack(all_segments)  # (N, 2, T)
all_labels = torch.tensor(all_labels, dtype=torch.long)

save_path = "/content/drive/MyDrive/sleep_segments/merged_sleep_dataset.pt"
torch.save((all_segments, all_labels), save_path)
print(f"✅ 병합 완료! 저장 위치: {save_path}")

🔄 피클 파일 병합 중...


<ipython-input-4-3bdd17a6ae7d>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  tensor_seg = torch.tensor(seg, dtype=torch.float32)


✅ 병합 완료! 저장 위치: /content/drive/MyDrive/sleep_segments/merged_sleep_dataset.pt


## 데이터 불러오기

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np

class SleepTensorDataset(Dataset):
    def __init__(self, data_tensor, label_tensor):
        self.data = data_tensor
        self.labels = label_tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 🔹 불러오기
merged_path = "/content/drive/MyDrive/sleep_segments/merged_sleep_dataset.pt"
print("🔄 병합된 데이터 불러오는 중...")
all_data, all_labels = torch.load(merged_path)
print(f"✅ 불러오기 완료: 총 샘플 = {len(all_data)}")

# 🔹 Train/Test Split
indices = np.arange(len(all_data))
np.random.seed(42)
np.random.shuffle(indices)
split = int(len(indices) * 0.8)
train_idx, test_idx = indices[:split], indices[split:]

train_dataset = SleepTensorDataset(all_data[train_idx], all_labels[train_idx])
test_dataset  = SleepTensorDataset(all_data[test_idx], all_labels[test_idx])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ DataLoader 생성 완료: Train = {len(train_dataset)} / Test = {len(test_dataset)}")

🔄 병합된 데이터 불러오는 중...
✅ 불러오기 완료: 총 샘플 = 326557
✅ DataLoader 생성 완료: Train = 261245 / Test = 65312


## 학습 파이프라인

## 학습/평가 정의

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x, y in tqdm(dataloader, desc="Train"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output, _ = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        preds = output.argmax(1)
        total_loss += loss.item()
        correct += (preds == y).sum().item()
        total += y.size(0)

    acc = correct / total * 100
    return total_loss / len(dataloader), acc

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

def evaluate(model, dataloader, criterion, device):
    model.eval()
    y_true, y_pred = [], []
    total_loss = 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            output, _ = model(x)
            loss = criterion(output, y)
            total_loss += loss.item()

            preds = output.argmax(dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred, average='macro')
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')

    print("\n📊 Classification Report:\n")
    print(classification_report(y_true, y_pred, digits=3))

    return total_loss / len(dataloader), acc, f1, prec, rec

## 클래스 균등

In [ ]:
x_tensor, y_tensor = torch.load('/content/drive/MyDrive/sleep_segments/merged_sleep_dataset.pt')

from torch.utils.data import TensorDataset, random_split, DataLoader, WeightedRandomSampler

dataset = TensorDataset(x_tensor, y_tensor)
train_len = int(len(dataset) * 0.8)
train_set, test_set = random_split(dataset, [train_len, len(dataset) - train_len])

# --- 1번 전략: WeightedRandomSampler 적용 ---
train_indices = train_set.indices if hasattr(train_set, 'indices') else train_set._indices
train_labels = y_tensor[train_indices]  # 라벨 추출

class_counts = torch.bincount(train_labels)
class_weights = 1.0 / class_counts.float()
sample_weights = class_weights[train_labels]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(train_labels), replacement=True)

# --- DataLoader 생성 ---
train_loader = DataLoader(train_set, batch_size=16, sampler=sampler)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

print(f"DataLoader 생성 완료: Train = {len(train_set)}개 / Test = {len(test_set)}개")

✅ DataLoader 생성 완료: Train = 261245개 / Test = 65312개


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm import tqdm

In [ ]:
import numpy as np
import torch.nn.functional as F

def cb_loss(logits, labels, samples_per_cls, beta=0.9999, gamma=2.0):
    """
    logits: [B, C]
    labels: [B]
    samples_per_cls: list or array of class frequencies

    """
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * len(samples_per_cls)

    weights = torch.tensor(weights).float().to(logits.device)
    labels_one_hot = F.one_hot(labels, num_classes=logits.size(1)).float()
    weights = (weights.unsqueeze(0) * labels_one_hot).sum(1)

    ce = F.cross_entropy(logits, labels, reduction='none')  # shape: (B,)
    pt = torch.exp(-ce)
    loss = weights * ((1 - pt) ** gamma) * ce
    return loss.mean()

In [ ]:
# 클래스별 샘플 수 정의
samples_per_cls = [44965, 3010, 10696, 2323, 4318]

# 손실 함수 정의
criterion = lambda logits, labels: cb_loss(logits, labels, samples_per_cls=samples_per_cls)

# 모델, 옵티마이저
model = CSTAN_TCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 기록
train_losses, val_losses, val_accuracies = [], [], []
all_f1, all_precision, all_recall = [], [], []

best_val_loss = float('inf')
patience = 3
patience_counter = 0
best_model_state = None

for epoch in range(1, 41):
    print(f"\n🌀 Epoch {epoch}")

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    val_loss, val_acc, f1, prec, rec = evaluate(model, test_loader, criterion, device)
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | F1: {f1:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f}")

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    all_f1.append(f1)
    all_precision.append(prec)
    all_recall.append(rec)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\n⏹️ Early stopping at epoch {epoch}. Best Val Loss: {best_val_loss:.4f}")
            break

# 가장 좋은 모델 복원
if best_model_state is not None:
    model.load_state_dict(best_model_state)

# 최종 성능 요약
print("\n📊 최종 모델 평가 요약:")
print(f"Average Validation Accuracy: {np.mean(val_accuracies):.2f}%")
print(f"Average Macro F1 Score: {np.mean(all_f1):.4f}")
print(f"Average Precision: {np.mean(all_precision):.4f}")
print(f"Average Recall: {np.mean(all_recall):.4f}")
print(f"Last Epoch Accuracy: {val_accuracies[-1]:.2f}% | F1: {all_f1[-1]:.4f} | Precision: {all_precision[-1]:.4f} | Recall: {all_recall[-1]:.4f}")


🌀 Epoch 1


Train: 100%|██████████| 16328/16328 [03:16<00:00, 83.14it/s]


Train Loss: 0.3004 | Train Accuracy: 69.36%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


📊 Classification Report:

              precision    recall  f1-score   support

           0      1.000     0.000     0.000     45060
           1      0.045     1.000     0.086      2946
           2      0.000     0.000     0.000     10716
           3      0.000     0.000     0.000      2444
           4      0.000     0.000     0.000      4146

    accuracy                          0.045     65312
   macro avg      0.209     0.200     0.017     65312
weighted avg      0.692     0.045     0.004     65312

Val Loss: 16.8640 | Val Acc: 4.52% | F1: 0.017 | Precision: 0.209 | Recall: 0.200

🌀 Epoch 2


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


📊 Classification Report:

              precision    recall  f1-score   support

           0      0.690     1.000     0.817     45060
           1      0.000     0.000     0.000      2946
           2      0.000     0.000     0.000     10716
           3      0.000     0.000     0.000      2444
           4      0.000     0.000     0.000      4146

    accuracy                          0.690     65312
   macro avg      0.138     0.200     0.163     65312
weighted avg      0.476     0.690     0.563     65312

Val Loss: 15.2519 | Val Acc: 68.99% | F1: 0.163 | Precision: 0.138 | Recall: 0.200

🌀 Epoch 5


Train: 100%|██████████| 16328/16328 [03:19<00:00, 81.87it/s]


Train Loss: 0.1623 | Train Accuracy: 80.75%

📊 Classification Report:

              precision    recall  f1-score   support

           0      0.690     0.999     0.816     45060
           1      0.000     0.000     0.000      2946
           2      0.000     0.000     0.000     10716
           3      0.000     0.000     0.000      2444
           4      0.000     0.000     0.000      4146

    accuracy                          0.689     65312
   macro avg      0.138     0.200     0.163     65312
weighted avg      0.476     0.689     0.563     65312

Val Loss: 20.5776 | Val Acc: 68.91% | F1: 0.163 | Precision: 0.138 | Recall: 0.200

⏹️ Early stopping at epoch 5. Best Val Loss: 14.8375

📊 최종 모델 평가 요약:
Average Validation Accuracy: 56.08%
Average Macro F1 Score: 0.1341
Average Precision: 0.1522
Average Recall: 0.2000
Last Epoch Accuracy: 68.91% | F1: 0.1632 | Precision: 0.1379 | Recall: 0.1998


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
print("\n🧪 테스트 세트 성능 평가:")
test_loss, test_acc, test_f1, test_prec, test_rec = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f} | Accuracy: {test_acc:.2f}%")
print(f"Macro F1: {test_f1:.4f} | Precision: {test_prec:.4f} | Recall: {test_rec:.4f}")


🧪 테스트 세트 성능 평가:

📊 Classification Report:

              precision    recall  f1-score   support

           0      0.690     0.999     0.816     45060
           1      0.000     0.000     0.000      2946
           2      0.000     0.000     0.000     10716
           3      0.000     0.000     0.000      2444
           4      0.000     0.000     0.000      4146

    accuracy                          0.689     65312
   macro avg      0.138     0.200     0.163     65312
weighted avg      0.476     0.689     0.563     65312

Test Loss: 20.5776 | Accuracy: 68.91%
Macro F1: 0.1632 | Precision: 0.1379 | Recall: 0.1998


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
import torch
import pickle

# 모델 학습이 끝난 후 (예: best 모델 기준)
model_path_pt = '/content/drive/MyDrive/best_model.pt'
model_path_pkl = '/content/drive/MyDrive/best_model.pkl'

# 저장
torch.save(model.state_dict(), model_path_pt)

# 저징 : pkl
with open(model_path_pkl, 'wb') as f:
    pickle.dump(model.state_dict(), f)

print("모델 저장 완료 (.pt, .pkl)")

✅ 모델 저장 완료 (.pt, .pkl)


##시각화

In [ ]:
# 평가 모드
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Evaluating"):
        x = x.to(device)
        out, _ = model(x)
        preds = out.argmax(dim=1).cpu()
        y_pred.extend(preds.tolist())
        y_true.extend(y.tolist())

# 평가 지표 출력
print("✅ 평가 결과")
print("Accuracy      :", round(accuracy_score(y_true, y_pred) * 100, 2), "%")
print("Macro F1      :", round(f1_score(y_true, y_pred, average="macro") * 100, 2), "%")
print("Macro Precision:", round(precision_score(y_true, y_pred, average="macro") * 100, 2), "%")
print("Macro Recall  :", round(recall_score(y_true, y_pred, average="macro") * 100, 2), "%")

Evaluating: 100%|██████████| 4082/4082 [00:18<00:00, 223.33it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


✅ 평가 결과
Accuracy      : 68.8 %
Macro F1      : 16.3 %
Macro Precision: 13.77 %
Macro Recall  : 19.99 %
